In [1]:
import json
from dateutil.parser import parse
from datetime import timedelta
import requests
import math
import unicodecsv as csv
import numpy

In [2]:
"""
* house: whether or not the legislator is a member of the House of Representatives at the time of the statement. 1 indicates they are a member of the House, 0 indicates they are a member of the Senate. Blank values indicate that the data is either not known or that they are a member of neither (speaker, etc).
* sex: the sex of the legislator, according to the United States legislator biography project.
* last_name: the last name of the legislator who made the statement.
* dim_1: the 1st dimension DW-NOMINATE constant space score for the legislator.
* dim_2: the 2nd dimension DW-NOMINATE constant space score for the legislator.
* id: the ID of the statement (calculated by finding the MD5 hash of the date, text, document title, and speaker last name appended to one another, see `annotate.py` for implementation).
* first_name: the first name of the legislator.
* icpsr: the ICPSR legislator code.
* state: the first seven letters of the legislator’s state.
* statement: the raw text of the statement, as recorded in the Congressional Record.
* party: the legislator’s party. 1 indicates Democrats, 0 indicates Republicans.
* congress: the Congress that the statement was made in.
* days_until_term_ends: the number of days until the legislator is up for reelection.
* loyalty: whether the legislator’s party is the same as the President. 1 if true, 0 otherwise.
* date: the date the statement was made.
* state_code: the ICPSR state code.
* votes: the number of votes considered in the DW-NOMINATE score of the legislator
* periphery: the standard deviations from the party norm, where positive values indicate more partisan tendencies and negative values indicate more moderate tendencies
* unsigned_periphery: the unsigned (absolute value) periphery score
* seniority: the number of congresses the legislator has served in, including the current congress, at the time of the statement
"""

'\n* house: whether or not the legislator is a member of the House of Representatives at the time of the statement. 1 indicates they are a member of the House, 0 indicates they are a member of the Senate. Blank values indicate that the data is either not known or that they are a member of neither (speaker, etc).\n* sex: the sex of the legislator, according to the United States legislator biography project.\n* last_name: the last name of the legislator who made the statement.\n* dim_1: the 1st dimension DW-NOMINATE constant space score for the legislator.\n* dim_2: the 2nd dimension DW-NOMINATE constant space score for the legislator.\n* id: the ID of the statement (calculated by finding the MD5 hash of the date, text, document title, and speaker last name appended to one another, see `annotate.py` for implementation).\n* first_name: the first name of the legislator.\n* icpsr: the ICPSR legislator code.\n* state: the first seven letters of the legislator\xe2\x80\x99s state.\n* statement

In [3]:
def get_congress(time):
    time = time - timedelta(days=3)
    year = time.year
    return int(math.ceil(0.5*year - 894))

In [4]:
# ADD DW-NOMINATE TO THE DATASET

def decode_dw_row(row):
    # decodes row such as:
    #  114   29774  71  24  CALIFOR    100  0  1   CAPPS         -0.389   -0.227      -81.84920   1077     26    0.927
    # format: congress, icpsr, state code, district number (0 if senate or president), state name, party code (100=dem, 200=republican), occupancy, office attainment type, name, 1st dimension coord, 2nd dimension coord, log likelyhood, # of votes, # of classification errors, geometric mean probability
    elements = [element.strip() for element in row.split("  ") if element.strip() != ""]
    if len(elements) == 16:
        del elements[9]
    if len(elements) != 15:
        print "error: != 15 elements in a row! {}".format(len(elements))
        print row
        raise Exception("!= 15 elements in row")
    return {
        "latest_congress": elements[0],
        "icpsr": elements[1],
        "state_code": elements[2],
        "district_number": elements[3],
        "state": elements[4],
        "party_code": int(elements[5]),
        "occupancy": elements[6],
        "office_attainment_type": elements[7],
        "last_name": elements[8].split(" ")[0],
        "dim_1": float(elements[9]),
        "dim_2": float(elements[10]),
        "log_likelyhood": float(elements[11]),
        "votes": int(elements[12]),
        "classification_errors": float(elements[13]),
        "geometric_mean_probability": float(elements[14])
    }
dw_nominate_scores = {}
raw_legislators = []
_already_included_legislators = []
minimum_congress = 106
terms = {}
with open("/Users/miles/Source/combine-cr/data/DW-NOMINATE.txt", "r") as dw:
    for row in dw.readlines():
        dat = decode_dw_row(row)
        party = ""
        if dat['party_code'] == 100:
            party = "Democrat"
        if dat['party_code'] == 200:
            party = "Republican"
        dat["party"] = party
        dat['last_name'] = dat["last_name"].replace("-", "")
        name = dat['last_name'][:8] + dat['state'] + str(dat['latest_congress'])
        print name
        dw_nominate_scores[name.lower()] = dat
        if dat['icpsr'] not in _already_included_legislators and dat['latest_congress'] >= minimum_congress:
            raw_legislators.append(dat)
            _already_included_legislators.append(dat['icpsr'])
        
        if dat['icpsr'] not in terms:
            terms[dat['icpsr']] = []
        if int(dat['latest_congress']) not in terms[dat['icpsr']]:
            terms[dat['icpsr']].append(int(dat['latest_congress']))

print len(raw_legislators)

STURGESCONNECT1
WADSWORTCONNECT1
SHERMANCONNECT1
HUNTINGTCONNECT1
TRUMBULLCONNECT1
VININGDELAWAR1
JACKSONGEORGIA1
BALDWINGEORGIA1
MATHEWSGEORGIA1
STONEMARYLAN1
SENEYMARYLAN1
CONTEEMARYLAN1
SMITHMARYLAN1
GALEMARYLAN1
CARROLLMARYLAN1
AMESMASSACH1
GOODHUEMASSACH1
GERRYMASSACH1
SEDGWICKMASSACH1
PARTRIDGMASSACH1
THACHERMASSACH1
LEONARDMASSACH1
GROUTMASSACH1
GILMANNEW HAM1
FOSTERNEW HAM1
LIVERMORNEW HAM1
BOUDINOTNEW JER1
CADWALADNEW JER1
SCHUREMANEW JER1
SINNICKSNEW JER1
FLOYDNEW YOR1
LAURANCENEW YOR1
BENSONNEW YOR1
HATHORNNEW YOR1
SILVESTENEW YOR1
VANRENSSNEW YOR1
ASHENORTH C1
WILLIAMSNORTH C1
BLOODWORNORTH C1
STEELENORTH C1
SEVIERNORTH C1
HIESTERPENNSYL1
MUHLENGBPENNSYL1
HARTLEYPENNSYL1
SCOTTPENNSYL1
WYNKOOPPENNSYL1
CLYMERPENNSYL1
FITZSIMOPENNSYL1
SMITHSOUTH C1
BURKESOUTH C1
HUGERSOUTH C1
SUMTERSOUTH C1
TUCKERSOUTH C1
WHITEVIRGINI1
BROWNVIRGINI1
MOOREVIRGINI1
LEEVIRGINI1
MADISONVIRGINI1
COLESVIRGINI1
PAGEVIRGINI1
PARKERVIRGINI1
BLANDVIRGINI1
GRIFFINVIRGINI1
JOHNSONCONNECT1
ELLSWORTCONNECT1

MERIWETHGEORGIA8
BRYANGEORGIA8
EARLYGEORGIA8
HAMMONDGEORGIA8
LYONKENTUCK8
BOYLEKENTUCK8
WALTONKENTUCK8
SANDFORDKENTUCK8
FOWLERKENTUCK8
BEDINGERKENTUCK8
CAMPBELLMARYLAN8
BOWIEMARYLAN8
PLATERMARYLAN8
MOOREMARYLAN8
MCCREERYMARYLAN8
ARCHERMARYLAN8
NICHOLSOMARYLAN8
DENNISMARYLAN8
EUSTISMASSACH8
CROWNINSMASSACH8
CUTLERMASSACH8
VARNUMMASSACH8
DWIGHTMASSACH8
TAGGARTMASSACH8
MITCHELLMASSACH8
WILLIAMSMASSACH8
BISHOPMASSACH8
HASTINGSMASSACH8
STEDMANMASSACH8
SKINNERMASSACH8
LARNEDMASSACH8
SEAVERMASSACH8
CUTTSMASSACH8
WADSWORTMASSACH8
THATCHERMASSACH8
CLAGETTNEW HAM8
TENNEYNEW HAM8
HOUGHNEW HAM8
BETTONNEW HAM8
HUNTNEW HAM8
HELMSNEW JER8
ELMERNEW JER8
MOTTNEW JER8
SLOANNEW JER8
SOUTHARDNEW JER8
BOYDNEW JER8
SMITHNEW YOR8
RIKERNEW YOR8
SANDSNEW YOR8
MITCHILLNEW YOR8
VANCORTLNEW YOR8
MCCORDNEW YOR8
VERPLANCNEW YOR8
HASBROUCNEW YOR8
LIVINGSTNEW YOR8
VANRENSSNEW YOR8
TIBBITTSNEW YOR8
PALMERNEW YOR8
THOMASNEW YOR8
SAMMONSNEW YOR8
ROOTNEW YOR8
GRISWOLDNEW YOR8
PATTERSONEW YOR8
PHELPSNEW YOR8
WYNNSNORTH C8

LOVETTNEW YOR13
MOFFITTNEW YOR13
TAYLORNEW YOR13
WINTERNEW YOR13
SHIPHERDNEW YOR13
BOYDNEW YOR13
MARKELLNEW YOR13
THOMPSONNEW YOR13
WILLIAMSNEW YOR13
MILLERNEW YOR13
SMITHNEW YOR13
KENTNEW YOR13
GEDDESNEW YOR13
AVERYNEW YOR13
COMSTOCKNEW YOR13
HOPKINSNEW YOR13
HOWELLNEW YOR13
MURFREENORTH C13
ALSTONNORTH C13
KENNEDYNORTH C13
GASTONNORTH C13
KINGNORTH C13
MACONNORTH C13
CULPEPPENORTH C13
STANFORDNORTH C13
YANCYNORTH C13
PEARSONNORTH C13
FORNEYNORTH C13
PICKENSNORTH C13
FRANKLINNORTH C13
MCOHIO13
ALEXANDEOHIO13
CREIGHTOOHIO13
CALDWELLOHIO13
KILBOURNOHIO13
BEALLOHIO13
CLENDENIOHIO13
SEYBERTPENNSYL13
INGERSOLPENNSYL13
ANDERSONPENNSYL13
CONARDPENNSYL13
DAVISPENNSYL13
ROBERTSPENNSYL13
HENDERSOPENNSYL13
WHITEHILPENNSYL13
GLONINGEPENNSYL13
CROUCHPENNSYL13
SLAYMAKEPENNSYL13
GLASGOWPENNSYL13
CRAWFORDPENNSYL13
REAPENNSYL13
INGHAMPENNSYL13
BROWNPENNSYL13
HYNEMANPENNSYL13
UDREEPENNSYL13
PIPERPENNSYL13
BARDPENNSYL13
IRWINPENNSYL13
SMITHPENNSYL13
FINDLEYPENNSYL13
LYLEPENNSYL13
GRIFFINPENNSYL13
TANNEH

BALDWINPENNSYL16
MOOREPENNSYL16
EDDYRHODE I16
HAZARDRHODE I16
PINCKNEYSOUTH C16
LOWNDESSOUTH C16
ERVINSOUTH C16
OVERSTRESOUTH C16
TUCKERSOUTH C16
SIMKINSSOUTH C16
EARLESOUTH C16
MCCREARYSOUTH C16
BREVARDSOUTH C16
RHEATENNESS16
COCKETENNESS16
JONESTENNESS16
ALLENTENNESS16
CANNONTENNESS16
BRYANTENNESS16
MEECHVERMONT16
RICHVERMONT16
STRONGVERMONT16
RICHARDSVERMONT16
CRAFTSVERMONT16
MALLARYVERMONT16
JACKSONVIRGINI16
PINDALLVIRGINI16
VANVIRGINI16
WILLIAMSVIRGINI16
MCCOYVIRGINI16
FLOYDVIRGINI16
SMYTHVIRGINI16
SMITHVIRGINI16
MERCERVIRGINI16
BALLVIRGINI16
MOOREVIRGINI16
STROTHERVIRGINI16
BARBOURVIRGINI16
GARNETTVIRGINI16
PARKERVIRGINI16
BURWELLVIRGINI16
TUCKERVIRGINI16
RANDOLPHVIRGINI16
ARCHERVIRGINI16
PLEASANTVIRGINI16
ALEXANDEVIRGINI16
JONESVIRGINI16
JOHNSONVIRGINI16
GRAYVIRGINI16
NEWTONVIRGINI16
NELSONVIRGINI16
TYLERVIRGINI16
WALKERALABAMA16
KINGALABAMA16
LANMANCONNECT16
DANACONNECT16
HORSEYDELAWAR16
VANDYKEDELAWAR16
WALKERGEORGIA16
ELLIOTTGEORGIA16
EDWARDSILLINOI16
THOMASILLINOI16
TAYLORIN

BROWNMARYLAN21
MITCHELLMARYLAN21
SPENCERMARYLAN21
WILSONMARYLAN21
GORHAMMASSACH21
CROWNINSMASSACH21
VARNUMMASSACH21
EVERETTMASSACH21
DAVISMASSACH21
KENDALLMASSACH21
GRENNELLMASSACH21
BATESMASSACH21
DWIGHTMASSACH21
BAILEYMASSACH21
RICHARDSMASSACH21
HODGESMASSACH21
REEDMASSACH21
HINDSMISSISS21
PETTISMISSOUR21
HUBBARDNEW HAM21
CHANDLERNEW HAM21
HARVEYNEW HAM21
HAMMONSNEW HAM21
BRODHEADNEW HAM21
WEEKSNEW HAM21
HUGHESNEW JER21
RANDOLPHNEW JER21
COOPERNEW JER21
PIERSONNEW JER21
CONDICTNEW JER21
SWANNEW JER21
LENTNEW YOR21
CROCHERONEW YOR21
CAMBRELENEW YOR21
VERPLANCNEW YOR21
WHITENEW YOR21
COWLESNEW YOR21
BOCKEENEW YOR21
CRAIGNEW YOR21
EAGERNEW YOR21
DEWITTNEW YOR21
STRONGNEW YOR21
DICKINSONEW YOR21
SPENCERNEW YOR21
KINGNEW YOR21
BORSTNEW YOR21
ANGELNEW YOR21
STORRSNEW YOR21
HOFFMANNEW YOR21
ARNOLDNEW YOR21
TAYLORNEW YOR21
MARTINDANEW YOR21
FINCHNEW YOR21
HAWKINSNEW YOR21
SANFORDNEW YOR21
MONELLNEW YOR21
BEEKMANNEW YOR21
EARLLNEW YOR21
POWERSNEW YOR21
MAXWELLNEW YOR21
ROSENEW YOR21
HALSEYNEW

...

TEJEDATEXAS104
GREENTEXAS104
JOHNSONTEXAS104
HANSENUTAH104
WAL./GREUTAH104
ORTONUTAH104
SANDERSVERMONT104
BATEMANVIRGINI104
PICKETTVIRGINI104
SCOTTVIRGINI104
SISISKYVIRGINI104
PAYNEVIRGINI104
GOODLATTVIRGINI104
BLILEYVIRGINI104
MORANVIRGINI104
BOUCHERVIRGINI104
WOLFVIRGINI104
DAVISVIRGINI104
WHITEWASHING104
METCALFWASHING104
SMITHWASHING104
HASTINGSWASHING104
NETHERCUWASHING104
DICKSWASHING104
MCDERMOTWASHING104
DUNNWASHING104
TATEWASHING104
MOLLOHANWEST VI104
WISEWEST VI104
RAHALLWEST VI104
NEUMANNWISCONS104
KLUGWISCONS104
GUNDERSOWISCONS104
KLECZKAWISCONS104
BARRETTWISCONS104
PETRIWISCONS104
OBEYWISCONS104
ROTHWISCONS104
SENSENBRWISCONS104
CUBINWYOMING104
HEFLINALABAMA104
SHELBYALABAMA104
MURKOWSKALASKA104
STEVENSALASKA104
KYLARIZONA104
MCCAINARIZONA104
BUMPERSARKANSA104
PRYORARKANSA104
BOXERCALIFOR104
FEINSTEICALIFOR104
BROWNCOLORAD104
CAMPBELLCOLORAD104
CAMPBELLCOLORAD104
DODDCONNECT104
LIEBERMACONNECT104
BIDENDELAWAR104
ROTHDELAWAR104
GRAHAMFLORIDA104
MACKFLORIDA104
COVERDELGEORGI

DOYLEPENNSYL106
GOODLINGPENNSYL106
MASCARAPENNSYL106
ENGLISHPENNSYL106
KENNEDYRHODE I106
WEYGANDRHODE I106
SANFORDSOUTH C106
SPENCESOUTH C106
GRAHAMSOUTH C106
DEMINTSOUTH C106
SPRATTSOUTH C106
CLYBURNSOUTH C106
THUNESOUTH D106
JENKINSTENNESS106
DUNCANTENNESS106
WAMPTENNESS106
HILLEARYTENNESS106
CLEMENTTENNESS106
GORDONTENNESS106
BRYANTTENNESS106
TANNERTENNESS106
FORDTENNESS106
SANDLINTEXAS106
TURNERTEXAS106
JOHNSONTEXAS106
HALLTEXAS106
SESSIONSTEXAS106
BARTONTEXAS106
ARCHERTEXAS106
BRADYTEXAS106
LAMPSONTEXAS106
DOGGETTTEXAS106
EDWARDSTEXAS106
GRANGERTEXAS106
THORNBERTEXAS106
PAULTEXAS106
HINOJOSATEXAS106
REYESTEXAS106
STENHOLMTEXAS106
JACKSONLTEXAS106
COMBESTTEXAS106
GONZALEZTEXAS106
SMITHTEXAS106
DELAYTEXAS106
BONILLATEXAS106
FROSTTEXAS106
BENTSENTEXAS106
ARMEYTEXAS106
ORTIZTEXAS106
RODRIGUETEXAS106
GREENTEXAS106
JOHNSONTEXAS106
HANSENUTAH106
COOKUTAH106
CANNONUTAH106
SANDERSVERMONT106
BATEMANVIRGINI106
PICKETTVIRGINI106
SCOTTVIRGINI106
SISISKYVIRGINI106
GOODEVIRGINI106
GOODEVIRGINI10

TIBERIOHIO108
BROWNOHIO108
LATOURETOHIO108
PRYCEOHIO108
REGULAOHIO108
RYANOHIO108
NEYOHIO108
SULLIVANOKLAHOM108
CARSONOKLAHOM108
LUCASOKLAHOM108
COLEOKLAHOM108
ISTOOKOKLAHOM108
WUOREGON108
WALDENOREGON108
BLUMENAUOREGON108
DEFAZIOOREGON108
HOOLEYOREGON108
BRADYPENNSYL108
FATTAHPENNSYL108
ENGLISHPENNSYL108
HARTPENNSYL108
PETERSONPENNSYL108
GERLACHPENNSYL108
WELDONPENNSYL108
GREENWOOPENNSYL108
SHUSTERPENNSYL108
SHERWOODPENNSYL108
KANJORSKPENNSYL108
MURTHAPENNSYL108
HOEFFELPENNSYL108
DOYLEPENNSYL108
TOOMEYPENNSYL108
PITTSPENNSYL108
HOLDENPENNSYL108
MURPHYPENNSYL108
PLATTSPENNSYL108
KENNEDYRHODE I108
LANGEVINRHODE I108
BROWNSOUTH C108
WILSONSOUTH C108
BARRETTSOUTH C108
DEMINTSOUTH C108
SPRATTSOUTH C108
CLYBURNSOUTH C108
JANKLOWSOUTH D108
HERSETHSOUTH D108
JENKINSTENNESS108
DUNCANTENNESS108
WAMPTENNESS108
DAVISTENNESS108
COOPERTENNESS108
GORDONTENNESS108
BLACKBURTENNESS108
TANNERTENNESS108
FORDTENNESS108
SANDLINTEXAS108
TURNERTEXAS108
JOHNSONTEXAS108
HALLTEXAS108
HALLTEXAS108
HENSARLITEXAS1

KAPTUROHIO110
KUCINICHOHIO110
JONESOHIO110
TIBERIOHIO110
SUTTONOHIO110
LATOURETOHIO110
PRYCEOHIO110
REGULAOHIO110
RYANOHIO110
SPACEOHIO110
SULLIVANOKLAHOM110
BORENOKLAHOM110
LUCASOKLAHOM110
COLEOKLAHOM110
FALLINOKLAHOM110
WUOREGON110
WALDENOREGON110
BLUMENAUOREGON110
DEFAZIOOREGON110
HOOLEYOREGON110
BRADYPENNSYL110
FATTAHPENNSYL110
ENGLISHPENNSYL110
ALTMIREPENNSYL110
PETERSONPENNSYL110
GERLACHPENNSYL110
SESTAKPENNSYL110
MURPHYPENNSYL110
SHUSTERPENNSYL110
CARNEYPENNSYL110
KANJORSKPENNSYL110
MURTHAPENNSYL110
SCHWARTZPENNSYL110
DOYLEPENNSYL110
DENTPENNSYL110
PITTSPENNSYL110
HOLDENPENNSYL110
MURPHYPENNSYL110
PLATTSPENNSYL110
KENNEDYRHODE I110
LANGEVINRHODE I110
BROWNSOUTH C110
WILSONSOUTH C110
BARRETTSOUTH C110
INGLISSOUTH C110
SPRATTSOUTH C110
CLYBURNSOUTH C110
HERSETHSOUTH D110
DAVISTENNESS110
DUNCANTENNESS110
WAMPTENNESS110
DAVISTENNESS110
COOPERTENNESS110
GORDONTENNESS110
BLACKBURTENNESS110
TANNERTENNESS110
COHENTENNESS110
GOHMERTTEXAS110
POETEXAS110
JOHNSONTEXAS110
HALLTEXAS110
HENSAR

HARTZLERMISSOUR112
CLEAVERMISSOUR112
GRAVESMISSOUR112
LONGMISSOUR112
EMERSONMISSOUR112
LUETKEMEMISSOUR112
REHBERGMONTANA112
FORTENBENEBRASK112
TERRYNEBRASK112
SMITHNEBRASK112
BERKLEYNEVADA112
HELLERNEVADA112
AMODEINEVADA112
HECKNEVADA112
GUINTANEW HAM112
BASSNEW HAM112
ANDREWSNEW JER112
LOBIONDONEW JER112
RUNYANNEW JER112
SMITHNEW JER112
GARRETTNEW JER112
PALLONENEW JER112
LANCENEW JER112
PASCRELLNEW JER112
ROTHMANNEW JER112
PAYNENEW JER112
PAYNENEW JER112
FRELINGHNEW JER112
HOLTNEW JER112
SIRESNEW JER112
HEINRICHNEW MEX112
PEARCENEW MEX112
LUJANNEW MEX112
BISHOPNEW YOR112
ISRAELNEW YOR112
KINGNEW YOR112
MCCARTHYNEW YOR112
ACKERMANNEW YOR112
MEEKSNEW YOR112
CROWLEYNEW YOR112
NADLERNEW YOR112
WEINERNEW YOR112
TURNERNEW YOR112
TOWNSNEW YOR112
CLARKENEW YOR112
VELAZQUENEW YOR112
GRIMMNEW YOR112
MALONEYNEW YOR112
RANGELNEW YOR112
SERRANONEW YOR112
ENGELNEW YOR112
LOWEYNEW YOR112
HAYWORTHNEW YOR112
GIBSONNEW YOR112
TONKONEW YOR112
HINCHEYNEW YOR112
OWENSNEW YOR112
HANNANEW YOR112
BUERKLENEW

In [5]:
mentions = {}
with open("/Users/miles/Permanent/ghraib.json", "r") as datain:
    for element in json.load(datain): # is array
        party_code = None
        if element['party'] == "Republican":
            party_code = 200
        elif element['party'] == "Democrat":
            party_code = 100
            
        dim_1 = None
        if element['dim_1'] is not None and element['dim_1'] != "Unknown":
            dim_1 = float(element['dim_1'])
    
        dim_2 = None
        if element['dim_2'] is not None and element['dim_2'] != "Unknown":
            dim_2 = float(element['dim_2'])

        
        icpsr = None
        district_number = None
        i = 8
        while icpsr is None and i > 0:
            if element['last_name'].lower() == "unknown":
                icpsr = None
                break
            congress = get_congress(parse(element['date']))
            if congress > 114:
                congress = 114 # must extrapolate
            name = element['last_name'][:i] + element['state'][:7] + str(congress).lower()
            try:
                icpsr = dw_nominate_scores[name.lower()]['icpsr']
                district_number = dw_nominate_scores[name.lower()]['district_number']
            except:
                print name
            i -= 1
        
        mentions[element['id']] = {
            "id": element['id'],
            "date": element['date'],
            "date_parsed": parse(element['date']),
            "title": element['title'],
            "first_name": element['first_name'],
            "last_name": element['last_name'],
            "party": element['party'],
            "sex": element['sex'],
            "state": element['state'],
            "statement": element['statement'],
            "congress": get_congress(parse(element['date'])),
            "days_until_term_ends": element['days_until_term_ends'],
            "dim_1": dim_1,
            "dim_2": dim_2,
            "party_code": party_code,
            "icpsr": icpsr,
            "district_number": district_number,
        }

RoybalAlCALIFOR108
RoybalAlCALIFOR108
BordalloGUAM109
BordallGUAM109
BordalGUAM109
BordaGUAM109
BordGUAM109
BorGUAM109
BoGUAM109
BGUAM109


In [6]:
def get_seniority(time, legislator):
    if 'icpsr' not in legislator or legislator['icpsr'] is None:
        return None
    congress = get_congress(time)
    terms_served = terms[legislator['icpsr']]
    count = 0
    for term in terms_served:
        if term <= congress:
            count += 1
    return count

def get_106_seniority(time, legislator):
    if 'icpsr' not in legislator or legislator['icpsr'] is None:
        return None
    congress = get_congress(time)
    terms_served = terms[legislator['icpsr']]
    count = 0
    for term in terms_served:
        if term <= congress and term >= 106:
            count += 1
    return count

In [7]:
republican_std_dev = numpy.std([legislator["dim_1"] for legislator in raw_legislators if legislator["party_code"] == 200])
democrat_std_dev = numpy.std([legislator["dim_1"] for legislator in raw_legislators if legislator["party_code"] == 100])

republican_mean = numpy.mean([legislator["dim_1"] for legislator in raw_legislators if legislator["party_code"] == 200])
democrat_mean = numpy.mean([legislator["dim_1"] for legislator in raw_legislators if legislator["party_code"] == 100])

def periphery(legislator):
    if legislator['dim_1'] is None or legislator['dim_2'] is None:
        return None
    
    if "party_code" not in legislator:
        return None # legislator not known
    if legislator["party_code"] not in [100, 200]:
        print legislator["party_code"]
        return None
    party_std_dev = None # not pythonic, not a problem
    party_mean = None
    opposite_mean = None
    if legislator["party_code"] is 200:
        party_std_dev = republican_std_dev
        party_mean = republican_mean
        opposite_mean = democrat_mean
    elif legislator["party_code"] is 100:
        party_std_dev = democrat_std_dev
        party_mean = democrat_mean
        opposite_mean = republican_mean
    
    score = abs(party_mean - legislator["dim_1"]) / party_std_dev
    
    neg = (legislator["dim_1"] < party_mean) == (opposite_mean < party_mean)
    if neg:
        score *= -1
        
    return score

In [8]:
print "Republican standard deviation: " + str(republican_std_dev)
print "Democrat standard deviation: " + str(democrat_std_dev)
print "Republican mean 1st dimension score: " + str(republican_mean)
print "Democrat mean 1st dimension score: " + str(democrat_mean)

Republican standard deviation: 0.225571467647
Democrat standard deviation: 0.172697888806
Republican mean 1st dimension score: 0.310150626769
Democrat mean 1st dimension score: -0.311623203078


In [9]:
from datetime import date
def get_current_executive_party(statement_date):
    if statement_date > date(2017,1,19):
        return 0
    if statement_date > date(2009,1,19):
        return 1
    if statement_date > date(2001, 1, 19):
        return 0
    return None

In [10]:
import json
raw_classifications = []
with open("../data/ghraib_restraining.json", "r") as infile:
    raw_classifications = json.load(infile)

In [11]:
classifications = {}
for entry in raw_classifications:
    classifications[entry['id']] = int(entry['restraining'])

In [12]:
# normalize data that is not already normalized

mentions_compiled = []
for z in mentions.itervalues():
    
#     if z['id'] in addl_fields:
#         z['days_until_term_ends'] = addl_fields[z['id']]['days_until_term_ends']

    if z['id'] in classifications:
        z['restraining'] = classifications[z['id']]
    else:
        z['restraining'] = ""
        print z['id']

    # party scores
    try:
        if z['party'] == "Democrat":
            z['party'] = 1
        elif z['party'] == "Republican":
            z['party'] = 0
        else:
            z['party'] = None
    except:
        z['party'] = None
        
    # code sex into dataset (can't code gender, not binary, though neither is sex... oh well, STATA doesn't care...)
    try:
        if z['sex'] == "M":
            z['sex'] = 1
        elif z['sex'] == "F":
            z['sex'] = 0
        else:
            z['sex'] = None
    except:
        z['sex'] = None
        
    # absolute value perphery
    z['unsigned_periphery'] = None
    if 'periphery' in z and z['periphery'] is not None:
        z['unsigned_periphery'] = abs(z['periphery'])
        
    # loyalty
    if 'date_parsed' in z and 'party' in z:
        if z['party'] == get_current_executive_party(z['date_parsed'].date()):
            z['loyalty'] = 1
        else:
            z['loyalty'] = 0
    else:
        z['loyalty'] = None
            
    # party periphery
    z['periphery'] = periphery(z)
    if z['periphery'] is not None:
        z['unsigned_periphery'] = abs(z['periphery'])
    else:
        z['unsigned_periphery'] = None
        
    # house — somewhat unreliable for legislators who have served in both
    if 'dim_1' in z:
        if 'district_number' in z and z['district_number'] is not None:
            if int(z['district_number']) != 0:
                z['house'] = 1 # house
            else:
                z['house'] = 0 # senate
    else:
        z['house'] = None
    
    if 'icpsr' in z and z['icpsr'] is not None:
        z['seniority'] = get_seniority(z['date_parsed'], z)
    
    # delete unneded variables
    try:
        del z['date_parsed']
        del z['classification_errors']
        del z['party_code']
        del z['district_number']
        del z['occupancy']
        del z['geometric_mean_probability']
        del z['log_likelyhood']
        del z['latest_congress']
        del z['office_attainment_type']
    except:
        pass
        
    mentions_compiled.append(z)

None
None
None
None
None
None
None
None
None


In [13]:
keys = []
for z in mentions_compiled:
    for key in z.iterkeys():
        if key not in keys:
            keys.append(key)
print keys

['last_name', 'restraining', 'house', 'sex', 'district_number', 'seniority', 'dim_2', 'id', 'dim_1', 'first_name', 'icpsr', 'title', 'state', 'periphery', 'statement', 'party', 'congress', 'loyalty', 'days_until_term_ends', 'party_code', 'date', 'unsigned_periphery']


In [14]:
import unicodecsv as csv
with open("data_combined_ghraib.csv", "wb") as outfile:
    writer = csv.writer(outfile)
    writer.writerow(keys)
    for z in mentions_compiled:
        row = []
        for key in keys:
            if key in z and z[key] != "Unknown":
                row.append(z[key])
            else:
                row.append(None)
        writer.writerow(row)